In [1]:
import numpy as npb

In [2]:
## I have tried the OOPS framework here to make the code re-usable and scalable.
## Assumptions:
# 1. Min period is assumed to be 12 when calculating EMA_12 and EMA_16 of the stock
# 2. Min Period of 9 is assumed when calculating MACD Signal
# 3. Trading Cost is expensed on the same day
# 4. Drawdown is calculated as (Trough - Peak)/Peak


# Generic Class and Function for the Mean Reversion strategy used for Question 6 to 10.
class Backtest:
    def __init__(self,data):
        self.data=data
        
#    def generate_signal(self):
        ##yet to write
        
    def setup_backtest(self, start_date, end_date, base_, tradingexp_):
        self.base=base_
        self.tradingexp = tradingexp_
        ## Write any processing that needs to be done on the data
#         self.data = pdr.get_data_yahoo(self.ticker, start = start_date, end = end_date)
#         self.data['EMA_12']=self.data['Adj Close'].ewm(span=12,min_periods=12,adjust=False).mean()
#         self.data['EMA_26']=self.data['Adj Close'].ewm(span=26,min_periods=12, adjust=False).mean()
#         self.data['MACD']= self.data['EMA_12'] - self.data['EMA_26']
#         self.data['MACD_Signal']=self.data['MACD'].ewm(span=9,min_periods=9,adjust=False).mean()
#         self.data['Position']=np.where(self.data['MACD']>self.data['MACD_Signal'],1,np.where(self.data['MACD']<self.data['MACD_Signal'],-1,0))
#         self.data['Buy_Sell']= self.data['Position'] - self.data['Position'].shift(1)
#         self.data['log_return']=np.log(self.data['Adj Close']/self.data['Adj Close'].shift(1))
#         self.data['Strategy_return']=self.data['Position'].shift(1)*self.data['log_return']
#         self.data['Expense']=np.where(abs(self.data['Buy_Sell'])> 0, self.tradingexp, 0)
#         self.data['Capital']= self.base*self.data['Strategy_return'].cumsum().apply(np.exp)
#         self.data['Adj Capital'] = self.base*self.data['Strategy_return'].cumsum().apply(np.exp) - self.data['Expense'].shift(1).cumsum()
#         self.data['Adj return'] = np.log(self.data['Adj Capital']/self.data['Adj Capital'].shift(1))
#         self.data['drawdown']=self.data['Adj return'].cumsum() - self.data['Adj return'].cumsum().cummax()
        
    def plot_backtest_buy_sell(self): 
        ax=\
        (
            self.data[['Adj Close',
                       'Position']]
            .plot(figsize=(18,8),
                  secondary_y=["Position"],
                  style=["red","b--"],
                  alpha=0.5
                 )
        )

        (
            ax.legend(loc="upper center"
                      ,shadow=True
                      ,fancybox=True
                      ,bbox_to_anchor=(0.55,1.10)
                      ,ncol=4
                     )
        )

        bx=\
        (
            self.data[self.data['Buy_Sell']>0]['Adj Close']
            .plot(style="g^",
                  markersize=12,
                  label='Buy')
        )

        bx.legend()

        cx=\
        (
            self.data[self.data['Buy_Sell']<0]['Adj Close']
            .plot(style="rv",
                  markersize=12,
                  label="Sell")
        )
        
        cx.legend()
        plt.title(f"Backtest Buy Sell Signals for {self.ticker}")
        plt.show()
      
    def calculate_cumm_return(self):
        res=100*(np.exp(self.data['Adj return'].sum()) -1)
        print(f"Cummulative return for {self.ticker} is: {res:.2f}%")

    def calculate_max_drawdown(self):
        res=100* (np.exp(self.data['drawdown'].min()) - 1)
        print(f"{self.ticker} Max Drawdown: {res:.2f}%")
    
    def calculate_max_drawdown_duration(self):
        periods =\
        self.data[self.data['drawdown'] == 0].index[ 1 :   ].to_pydatetime() \
        - self.data[self.data['drawdown'] == 0].index[   : -1].to_pydatetime()
        print(f"{self.ticker} Max Drawdown Duration: {periods.max()}")

    def calculate_return_byyear(self):
        grpByYear=self.data.groupby(self.data.index.year)
        return grpByYear['Adj return'].sum().apply(np.exp) -1

    def calculate_sharpe_ratio(self):
        res = np.sqrt(252) * (self.data['Adj return'].dropna().apply(np.exp).add(-1).mean()) / (self.data['Adj return'].dropna().apply(np.exp).add(-1).std())
        print(f"{self.ticker} Sharpe Ratio: {res:.3f}")
        
    def plot_backtest_returns(self):
        ax =\
        (
            self.data[["log_return", "Adj return"]]
            .dropna()
            .cumsum()
            .apply(np.exp)
            .add(-1)
            .plot(figsize = [18, 7])
        )
        
        ax.set_title(f"{self.ticker}: Back Test Strategy vs Asset Return")
        ax.legend(["Asset Cumm Return","Strategy Cumm Return (Adj)"])
        bx =\
        (
            self.data['Adj return']
            .cumsum()
            .cummax()
            .apply(np.exp)
            .add(-1)
            .plot(ax=ax)
        )
        plt.show()